<a href="https://colab.research.google.com/github/Datascientisit/ML_regression/blob/main/pnevmaniya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/drive/MyDrive/train.zip', 'r') as zip2:
    zip2.extractall('/content/drive/MyDrive/train_extracted')

In [ ]:
from fastai.vision.all import *
import pandas as pd
path = Path('/content/drive/MyDrive/train_extracted')
# path.ls()

path.ls()

In [ ]:
PNEUMONIA = (path/'PNEUMONIA').ls().sorted()
NORMAL = (path/'NORMAL').ls().sorted()
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
len(PNEUMONIA), len(NORMAL)

In [ ]:
n = len(NORMAL)
PNEUMONIA = PNEUMONIA[:n]
len(PNEUMONIA), len(NORMAL)

In [ ]:
imP_path = PNEUMONIA[1]
imP = Image.open(imP_path)
imP

In [ ]:
lungs = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224)
)

dls = lungs.dataloaders(path)

learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)

In [ ]:
path_test = Path('/content/drive/MyDrive/test')
# path.ls()

path_test.ls()

In [ ]:
image_files = list(Path(path_test).glob("*.jpeg"))
ind = 0
for item in sample["id"].values:
    img = PILImage.create(image_files[ind])
    pred, pred_id ,probs = learn.predict(img)
    sample.at[ind,"labels"] = pred
    ind += 1

In [ ]:
import os

img_path = "/content/drive/MyDrive/test/test_img_10238.jpeg"
print("File exists:", os.path.exists(img_path))
print("File size:", os.path.getsize(img_path), "bytes")


In [ ]:
import imghdr

print(imghdr.what(img_path))

In [ ]:
print("Total images found:", len(image_files))
print("First 5 images:", image_files[:5])

In [ ]:
from pathlib import Path

path_test = Path("/content/drive/MyDrive/test")
image_files = list(path_test.glob("*.jpeg"))

print("Total images found:", len(image_files))

In [ ]:
list(path_test.glob("*.*"))

In [ ]:
ind = 0
for item in sample["id"].values:
    img_path = image_files[ind]

    try:
        img = PILImage.create(img_path)
        pred, pred_id, probs = learn.predict(img)
        sample.at[ind, "labels"] = pred
    except Exception as e:
        print(f"Skipping {img_path} due to error: {e}")
    ind+=1
    if ind<=600:
      break

In [ ]:
inter = ClassificationInterpretation.from_learner(learn)
inter.plot_confusion_matrix()

In [ ]:
inter.print_classification_report()

In [ ]:
inter.plot_top_losses(10, nrows=5)